In [65]:
library(tidyverse)
library(UpSetR)

In [66]:
# Metadtata

metadata <- read.table("/home/yuliya/repos/cosybio/FedProt/data/TMT_data/raw_MaxQuant_reports/Metadata_CosyBio.tsv",
                       header = TRUE, sep = "\t", stringsAsFactors = FALSE)
                       
metadata <- metadata %>%
            mutate(Quantitative.column.name = gsub(" ", ".", Quantitative.column.name))

rownames(metadata) <- metadata$Quantitative.column.name
metadata <- metadata %>%
            mutate(Quantitative.column.name = gsub(".Pool", ".P_", gsub("Reporter.intensity.corrected.", "RIC_", Quantitative.column.name)))


# remove the outlier
metadata <- metadata %>%
            filter(Quantitative.column.name != "RIC_3.P_3")


# rename the columns - put the .P_[1-9]+ after dot before and the rest after (e.g. RIC_1.P_1 -> P_1.RIC_1)
metadata <- metadata %>%
            mutate(Quantitative.column.name = gsub("RIC_([0-9]+).P_([0-9]+)", "P_\\2.RIC_\\1", Quantitative.column.name))



head(metadata, 3)
dim(metadata)

,Quantitative.column.name,Pool,Reporter.ion,Patient,Group,Center
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Reporter.intensity.corrected.1.Pool1,P_1.RIC_1,Pool1,126,Common Reference,Common Reference,Center1
Reporter.intensity.corrected.2.Pool1,P_1.RIC_2,Pool1,127N,heathy1,heathy,Center1
Reporter.intensity.corrected.3.Pool1,P_1.RIC_3,Pool1,127C,FSGS1,FSGS,Center1


[1] 65  6

# Logic

Three types of data:
- (1) Data summarized using Gene names
- (2) Data summarized using Major.PG.IDs
- (3) Data summarized to PG from peptides level.
- (4) Data summarized to Gene.names from peptides level.


For (1) and (2) PG_group report is used.  
Filtered, then features names and samples columns (+ razor+unique peptides are extracted) and stored into separated files.

For (3) and (4) approach from MaxQuant for data aggregation is used. Filtered on peptides level, then summarized into PG group using MaxQuant logic. 

# Extract data for (1) and (2)

In [67]:
# Function to preprocess the data from a given path and metadata
# Args:
#   path: The file path of the data to be preprocessed
#   meta_data: The metadata associated with the data
# Returns:
#   A list containing the preprocessed protein intensities and counts
preprocess_mxout <- function(path, meta_data){
    PG_report <- read.table(
        path,
        header=T,
        sep="\t",
        stringsAsFactors = F)
    
    # remove decoy matches and matches to contaminant
    PG_report <- PG_report[!PG_report$Reverse=="+",]
    PG_report <- PG_report[!PG_report$Potential.contaminant=="+",]
    # if the PG_report$Only.identified.by.site is not all NA and contains "+" values - remove this rows
    if(!all(is.na(PG_report$Only.identified.by.site))) {
        # # Only.identified.by.site - exclude proteins that were only identified by a modification site
        PG_report <- PG_report[!PG_report$Only.identified.by.site=="+",]
    }

    # PG_report <- PG_report[PG_report$Gene.names != '',]


    pg_intensities <- PG_report %>%
        select(c(Majority.protein.IDs, Gene.names, rownames(meta_data)))
    colnames(pg_intensities) <- c("Majority.protein.IDs", "Gene.names", meta_data$Quantitative.column.name)
    print(length(pg_intensities$Majority.protein.IDs))

    # counts
    pg_counts <- PG_report %>%
        select(c("Majority.protein.IDs", "Gene.names", "Peptide.IDs", "Razor...unique.peptides", "Peptide.is.razor"))

    return(list(pg_intensities, pg_counts)) 
}

In [68]:
# List of input files
list_of_inputs <- list(
    "Center1" = "/home/yuliya/repos/cosybio/FedProt/data/TMT_data/raw_MaxQuant_reports/proteinGroups_center1.txt",
    "Center2" = "/home/yuliya/repos/cosybio/FedProt/data/TMT_data/raw_MaxQuant_reports/proteinGroups_center2.txt",
    "Center3" = "/home/yuliya/repos/cosybio/FedProt/data/TMT_data/raw_MaxQuant_reports/proteinGroups_center3.txt"
)

# Initialize empty lists to store combined data
combined_pg_intensities <- list()
combined_counts <- list()

# Iterate over each center
for(center in names(list_of_inputs)) {
    # Determine the center folder based on the center name
    center_folder <- ifelse(center == "Center1", "center_one", ifelse(center == "Center2", "center_two", "center_three"))

    # Filter metadata for the current center
    center_metadata <- metadata %>%
        filter(Center == center)

    # Preprocess the MaxQuant output file for the current center
    results_list <- preprocess_mxout(list_of_inputs[[center]], center_metadata)
    pg_intensities <- results_list[[1]]
    counts_df <- results_list[[2]]

    # Add the processed data to the combined lists
    combined_pg_intensities <- c(combined_pg_intensities, list(pg_intensities))
    combined_counts <- c(combined_counts, list(counts_df))
}

# Add names to the combined lists
names(combined_pg_intensities) <- names(list_of_inputs)
names(combined_counts) <- names(list_of_inputs)

[1] 500
[1] 530
[1] 456


### (1)

In [69]:
# Set the output path for the gene intensities and counts
genes_output_path <- "/home/yuliya/repos/cosybio/FedProt/data/TMT_data/balanced_data/01_PG_Genes/"

# Iterate over each center
for(center in list("Center1", "Center2", "Center3")) {
  
  # Summarize gene intensities
  # Calculate the sum of intensities for each gene and concatenate multiple gene names
  # separated by semicolons into a single string
  genes_intensities <- combined_pg_intensities[[center]] %>%
    select(-Majority.protein.IDs) %>%
    group_by(Gene.names) %>%
    summarize_all(sum) %>%
    ungroup() %>%
    filter(Gene.names != "") %>%
    rowwise() %>% # Apply the following operations to each row individually
    mutate(Gene.names = map_chr(strsplit(Gene.names, ";"), ~paste(sort(.x), collapse = ";")))

  # Write the gene intensities to a file
  write.table(genes_intensities, paste0(genes_output_path, center, "/genes_intensities.tsv"), sep = "\t", quote = FALSE, row.names = FALSE)
    
  # Summarize gene counts
  # Filter, select, and manipulate the counts dataframe
  "Peptide.is.razor"
  # counts_df <- combined_counts[[center]] %>%
  #   filter(Gene.names != "") %>%
  #   select(-c(Majority.protein.IDs, Razor...unique.peptides)) %>%
  #   mutate(Peptide.IDs = strsplit(Peptide.IDs, ";")) %>%
  #   unnest(Peptide.IDs) %>%  unique() %>%
  #   group_by(Gene.names) %>%  summarize(Unique.peptides = n()) %>%
  #   rowwise() %>% # Apply the following operations to each row individually
  #   mutate(Gene.names = map_chr(strsplit(Gene.names, ";"), ~paste(sort(.x), collapse = ";")))
  counts_df <- combined_counts[[center]] %>%
    filter(Gene.names != "") %>%
    select(-c(Majority.protein.IDs, Razor...unique.peptides)) %>%
    mutate(Peptide.IDs = strsplit(Peptide.is.razor, ";")) %>%
    unnest(Peptide.IDs) %>%  
    filter(Peptide.IDs == "True") %>%
    group_by(Gene.names) %>%  summarize(Unique.peptides = n()) %>%
    rowwise() %>% # Apply the following operations to each row individually
    mutate(Gene.names = map_chr(strsplit(Gene.names, ";"), ~paste(sort(.x), collapse = ";")))

  # This code performs data preparation steps on the counts dataframe. 
  #It filters out rows where the Gene.names column is empty, removes unnecessary columns, 
  # splits the Peptide.IDs column into separate values, removes duplicate rows, 
  # groups the data by Gene.names, calculates the count of unique peptides for each gene, 
  # and finally sorts and collapses the Gene.names values. 
  #The resulting dataframe is stored in the counts_df variable.

  # Write the gene counts to a file
  write.table(counts_df, paste0(genes_output_path, center, "/genes_counts_v2.tsv"), sep = "\t", quote = FALSE, row.names = FALSE)

  # Write the metadata to a file
  metadata %>%
    filter(Center == center) %>%
    write.table(paste0(genes_output_path, center, "/metadata.tsv"), sep = "\t", quote = FALSE, row.names = FALSE)
}

### (2)

In [70]:
pg_output_path <- "/home/yuliya/repos/cosybio/FedProt/data/TMT_data/balanced_data/02_PG_MajorPG/"

# This loop iterates over a list of centers and performs data preparation tasks for each center.
for(center in list("Center1", "Center2", "Center3")) {
  
  # Summarize gene names
  genes_intensities <- combined_pg_intensities[[center]] %>%
    select(-Gene.names) %>%
    group_by(Majority.protein.IDs) %>%
    rowwise() %>% # Apply the following operations to each row individually
    mutate(Majority.protein.IDs = map_chr(strsplit(Majority.protein.IDs, ";"), ~paste(sort(.x), collapse = ";")))

  # Write gene intensities to a file
  write.table(genes_intensities, paste0(pg_output_path, center, "/pg_intensities.tsv"), sep = "\t", quote = FALSE, row.names = FALSE)
    
  # Summarize counts
  counts_df <- combined_counts[[center]] %>%
    select(-c(Gene.names, Peptide.IDs, Peptide.is.razor)) %>%
    rowwise() %>% # Apply the following operations to each row individually
    mutate(Majority.protein.IDs = map_chr(strsplit(Majority.protein.IDs, ";"), ~paste(sort(.x), collapse = ";")))

  # Write counts to a file
  write.table(counts_df, paste0(pg_output_path, center, "/pg_counts.tsv"), sep = "\t", quote = FALSE, row.names = FALSE)

  # Write metadata to a file
  metadata %>%
    filter(Center == center) %>%
    write.table(paste0(pg_output_path, center, "/metadata.tsv"), sep = "\t", quote = FALSE, row.names = FALSE)
}

# Extract data for (3) and (4)

In [74]:
preprocess_mxout <- function(path, center_metadata, center){
    peptides_report <- read.table(
        path,
        header=T,
        sep="\t",
        stringsAsFactors = F)
    
    # remove decoy matches and matches to contaminant
    peptides_report <- peptides_report[!peptides_report$Reverse=="+",]
    peptides_report <- peptides_report[!peptides_report$Potential.contaminant=="+",]
    # if the peptides_report$Only.identified.by.site is not all NA and contains "+" values - remove this rows
    if(!all(is.na(peptides_report$Only.identified.by.site))) {
        # # Only.identified.by.site - exclude proteins that were only identified by a modification site
        peptides_report <- peptides_report[!peptides_report$Only.identified.by.site=="+",]
    }

    print(center)
    # take only the columns that we need
    pg_intensities <- peptides_report %>%
        select(c("Sequence", "Proteins", "Gene.names", rownames(center_metadata)))

    print(length(pg_intensities$Sequence))

    return(pg_intensities)
}

## (3) and (4) --- aggragated report

In [78]:
genes_output_path <- "/home/yuliya/repos/cosybio/FedProt/data/TMT_data/balanced_data/03_Peptides_Genes/"
pg_output_path <- "/home/yuliya/repos/cosybio/FedProt/data/TMT_data/balanced_data/04_Peptides_PG/"


list_of_outputs <- list(
    "Center1" = "/home/yuliya/repos/cosybio/FedProt/data/TMT_data/raw_MaxQuant_reports/Center1/peptides.txt",
    "Center2" = "/home/yuliya/repos/cosybio/FedProt/data/TMT_data/raw_MaxQuant_reports/Center2/peptides.txt",
    "Center3" = "/home/yuliya/repos/cosybio/FedProt/data/TMT_data/raw_MaxQuant_reports/Center3/peptides.txt"
)

combined_pg_intensities <- list()

for(center in names(list_of_outputs)) {

    center_metadata <- metadata %>%
        filter(Center == center)

    results_list <- preprocess_mxout(list_of_outputs[[center]], center_metadata, center)
    pg_intensities <- results_list

    write.table(pg_intensities, paste0(genes_output_path, center, "/aggregated.tsv"), sep = "\t", quote = FALSE, row.names = FALSE)
    write.table(pg_intensities, paste0(pg_output_path, center, "/aggregated.tsv"), sep = "\t", quote = FALSE, row.names = FALSE)
}


[1] "Center3"
[1] 4978


In [83]:
pg_intensities %>%
  select(-c(Proteins, rownames(center_metadata))) %>%
  filter(Gene.names != "") %>%
  #select rows where Gene.names doesn't contain ";"
  filter(!grepl(";", Gene.names)) 

Sequence,Gene.names
<chr>,<chr>
AAAEVAGQFVIK,TFRC
AAAPNTPK,PRG4
AAATGPSFWLGNETLK,PEPD
AAATLMSER,SLC4A1
AACLPLPGYR,LCP1
AADDTWEPFASGK,TTR
AAFCYQIR,MST1
AAFGQGSGPIMLDEVQCTGTEASLADCK,LGALS3BP
AAFNSGK,GAPDH
